In [10]:
## ROBT 310 - Final Project - Virtual Keyboard
## Team: Daryn Kalym, Alibek Manabayev, Alexey Muryshkin
## Date: April 27, 2018

%matplotlib inline

## modules
import numpy as np
from math import sqrt
import cv2
import sys

## global variables
frame = None
background = None
col_index_matrix = None

## functions

# Useless Function
def q(*x, **y):
    pass

# Blob Detection
def blobDetection(img):
    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 255;
    params.maxThreshold = 256;

    # Filter by Area.
    params.filterByArea = True
    params.minArea = 150

    # Filter by Circularity
#     params.filterByCircularity = True
#     params.minCircularity = 0.1

    # Filter by Convexity
#     params.filterByConvexity = True
#     params.minConvexity = 0.87

    # Filter by Inertia
#     params.filterByInertia = True
#     params.minInertiaRatio = 0.01
    
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs.
    keypoints = detector.detect( img )

    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    res = im_with_keypoints = cv2.drawKeypoints(img, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    return res

def getBoundingBox(img):
    temp, contours,hierarchy = cv2.findContours(img, 1, 2)
    cnt = contours[0]
    res = img
#     x,y,w,h = cv2.boundingRect(cnt)
#     res = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    
    return res

# Noise Filtering
def noiseFiltering(frame):
    res = np.uint8( frame )
    res = grayImage = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    res = cv2.equalizeHist(res)
    res = medFiltImage = cv2.medianBlur(res, 7)
    
    return res

def processFrame(frame):
    global background
    
    threshold = 30
    
    res = np.abs(np.int32(frame) - np.int32(background))
    
    mask = np.abs( np.int32(frame) - np.int32(background) ) > threshold
    backSubImage = np.zeros(frame.shape)
    backSubImage[mask] = 255
    res = backSubImage
    
# #     background = 0.7 * background + 0.3 * frame
    kernel = np.ones((7,7),np.uint8)
    erosion = cv2.erode(backSubImage, kernel, iterations = 1)
    kernel = np.ones((10,10),np.uint8)
    res = dilation = cv2.dilate(erosion, kernel, iterations = 1)
    #res = blobDetection( np.uint8(dilation) )
    #res = getBoundingBox( np.uint8(res) )
    
    return np.uint8( res )

def getCenterY(img):
    global col_index_matrix
    
    mask = img == 255
    n = np.sum(np.uint32(mask))
    ySum = np.sum( col_index_matrix[mask] )
    
    return int( round(ySum / n) ) if n else None
    
def displayVideoRealTime(cap):
    global frame, background, col_index_matrix
    
    ret, background = cap.read()
    background = noiseFiltering( np.flip( np.array(background, dtype=np.uint8), axis = 1 ) )
    col_index_matrix = np.array( [[j for j in range(background.shape[1])] for i in range(background.shape[0])] )
    
    o_window_name = "WebCam Video Streaming"
    cv2.namedWindow(o_window_name, flags=cv2.WINDOW_AUTOSIZE)
    cv2.moveWindow(o_window_name, 0, 0)
    
    p_window_name = "Processed Video Streaming"
    cv2.namedWindow(p_window_name, flags=cv2.WINDOW_AUTOSIZE)
    cv2.moveWindow(p_window_name, background.shape[1], 0)
    
    b_window_name = "Background Image"
    cv2.namedWindow(b_window_name, flags=cv2.WINDOW_NORMAL)
    cv2.resizeWindow(b_window_name, background.shape[1] // 3, background.shape[0] // 3)
    cv2.moveWindow(b_window_name, 0, 0)
    cv2.imshow(b_window_name, np.uint8( background ))
    
    while True:        
        ret, frame = cap.read()
        if not ret:
            break
            
        frame = noiseFiltering( np.flip( np.array(frame, dtype=np.uint8), axis = 1 ) )
        cv2.imshow(o_window_name, np.uint8( frame ))
        
        proc_img = processFrame(frame)
        #cv2.imshow(p_window_name, np.uint8( proc_img ))
        
        centerY = getCenterY(proc_img)
        if centerY is not None:
            proc_img[:, centerY] = 255
        cv2.imshow(p_window_name, np.uint8( proc_img ))
        
        key = cv2.waitKey(30)
        c = chr(key & 255)
        if c in ['q', 'Q', chr(27)]:
            break
        elif c in ['b', 'B']:
            background = frame
            cv2.imshow(b_window_name, np.uint8( background ))
    
    cap.release()
    cv2.destroyAllWindows()
    

def getVideoCapture(idd):
    if isinstance(idd, int):
        cap = cv2.VideoCapture( idd )
    else:
        cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print('Cannot initialize video capture with id {}'.format(idd))
        #sys.exit(-1)
        
    return cap

    
## main program
def main():
    devId = int( input('Please enter the id of the opened video capturing device:\n') )
    cap = getVideoCapture(devId)
    displayVideoRealTime(cap)

    
if __name__ == '__main__':
    main()


Please enter the id of the opened video capturing device:
0
